In [1]:
import numpy as np
import torch
import torch.nn as nn
train_data_path = "./data/matrix_data/train_data/"
file_list = os.listdir(train_data_path)
file_list.sort(key= lambda x:int(x[11:-4]))
train_data = []
for obj in file_list:   
    file_path = train_data_path + obj
    matrix = np.load(file_path)
    train_data.append(matrix)

In [2]:
train_data = np.array(train_data)
import torch
train_data = torch.from_numpy(train_data).float()
input = train_data[0]

In [3]:
conv = nn.Conv2d(3, 32, 3, (1, 1), 1)
output = conv(input)
print(output.shape)

conv2 = nn.Conv2d(32, 64, 3, (2, 2),1)
output2 = conv2(output)
print(output2.shape)

conv3 = nn.Conv2d(64, 128, 2, (2, 2), 1)
output3 = conv3(output2)
print(output3.shape)

conv4 = nn.Conv2d(128, 256, 2, (2, 2))
output4 = conv4(output3)
print(output4.shape)

torch.Size([5, 32, 30, 30])
torch.Size([5, 64, 15, 15])
torch.Size([5, 128, 8, 8])
torch.Size([5, 256, 4, 4])


In [8]:
from model.convolution_lstm import ConvLSTM
from torch.autograd import Variable
def attention(ConvLstm_out):
    attention_w = []
    for k in range(5):
            alpha = torch.mul(ConvLstm_out[k], ConvLstm_out[-1])/5
            m = nn.Softmax(dim=1)
            a = m(alpha).detach().numpy()
            attention_w.append(a)
    attention_w = torch.from_numpy(np.array(attention_w))
    out = torch.matmul(attention_w, ConvLstm_out).sum(0)
    out = torch.reshape(out, (1, out.shape[0], out.shape[1], out.shape[2]))
    return out

conv1_lstm = ConvLSTM(input_channels=32, hidden_channels=[32, 32, 32], kernel_size=3, step=5, effective_step=[4]).to("cpu")
conv1_lstm_out = conv1_lstm(output)
conv1_lstm_out = attention(conv1_lstm_out[0][0])
print(conv1_lstm_out.shape)

conv2_lstm = ConvLSTM(input_channels=64, hidden_channels=[64, 64, 64],  kernel_size=3, step=5, effective_step=[4]).to("cpu")
conv2_lstm_out = conv2_lstm(output2)
conv2_lstm_out = attention(conv2_lstm_out[0][0])
print(conv2_lstm_out.shape)

conv3_lstm = ConvLSTM(input_channels=128, hidden_channels=[128, 128, 128], kernel_size=3, step=5, effective_step=[4]).to("cpu")
conv3_lstm_out = conv3_lstm(output3)
conv3_lstm_out = attention(conv3_lstm_out[0][0])
print(conv3_lstm_out.shape)

conv4_lstm = ConvLSTM(input_channels=256, hidden_channels=[256, 256, 256], kernel_size=3, step=5, effective_step=[4]).to("cpu")
conv4_lstm_out = conv4_lstm(output4)
conv4_lstm_out = attention(conv4_lstm_out[0][0])
print(conv4_lstm_out.shape)

torch.Size([1, 32, 30, 30])
torch.Size([1, 64, 15, 15])
torch.Size([1, 128, 8, 8])
torch.Size([1, 256, 4, 4])


In [17]:
deconv4 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=[2, 2], padding=0)
deoutput4 = deconv4(conv4_lstm_out)
deconv4_concat = torch.cat((deoutput4, conv3_lstm_out), dim = 1)
# print(deconv4_concat.shape)

deconv3 = nn.ConvTranspose2d(in_channels=256, out_channels=64, kernel_size=3, stride=[2, 2], padding=1)
deoutput3 = deconv3(deconv4_concat)
deconv3_concat = torch.cat((deoutput3, conv2_lstm_out), dim = 1)
# print(deconv3_concat.shape)

deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=32, kernel_size=2, stride=[2, 2])
deoutput2 = deconv2(deconv3_concat)
deconv2_concat = torch.cat((deoutput2, conv1_lstm_out), dim = 1)
# print(deconv2_concat.shape)

deconv1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, stride=[1, 1], padding=1),
            nn.SELU()
        )
deoutput1 = deconv1(deconv2_concat)
print(deoutput1.shape)

torch.Size([1, 3, 30, 30])
